In [1]:
%load_ext gams_magic

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import *
from tensorflow.keras import backend as K
print(tf.__version__)

2.12.0


In [4]:
import sys
import os
import inspect

In [5]:
pwd = os.getcwd()
pardir = os.path.abspath(os.path.join(pwd, os.pardir))
print(pardir)

C:\Users\tk3016\OneDrive - Imperial College London\PhD


In [58]:
tanh_path = os.path.join(pwd,'Case2_bin_deep')
#relu_path = os.path.join(pardir,'relu_deep_model_ib_noT_miL2')
#rbf_path = os.path.join(pardir,'rbf_deep_model_ib_noT_miL2')

In [59]:
tanh_model = tf.keras.models.load_model(tanh_path, compile=False)
tanh_model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 7)                15        
 n)                                                              
                                                                 
 dense_76 (Dense)            (None, 10)                80        
                                                                 
 dense_77 (Dense)            (None, 7)                 77        
                                                                 
 dense_78 (Dense)            (None, 4)                 32        
                                                                 
 dense_79 (Dense)            (None, 1)                 5         
                                                                 
Total params: 209
Trainable params: 194
Non-trainable params: 15
______________________________________________________

In [60]:
weights = tanh_model.get_weights()

In [61]:
print(weights)

[array([  3.1303713 ,   3.1304212 ,   3.355535  ,   3.3560905 ,
         0.33328012,   0.33343843, 305.6317    ], dtype=float32), array([8.7556845e-01, 8.7662667e-01, 1.5693817e+00, 1.5718827e+00,
       3.2302596e-02, 3.2337684e-02, 5.2192368e+01], dtype=float32), 160563, array([[ 0.48758188,  0.47273085, -0.03927215, -1.0516748 ,  1.37077   ,
         0.8756834 ,  0.13668126,  0.97146744, -2.1373026 ,  0.40723315],
       [-2.310114  ,  1.3638446 , -0.09455798,  0.7336488 , -0.42569008,
        -0.3198279 , -1.3476624 ,  4.0151463 ,  1.2683836 , -0.58136517],
       [-0.8029893 ,  2.8575945 , -0.85216784,  1.7873511 , -3.1723704 ,
        -1.6707109 , -0.7943801 ,  0.17557839,  1.2999219 , -1.0089676 ],
       [ 2.440909  ,  1.9435128 ,  1.4965845 , -1.0980355 ,  0.45423198,
        -0.7906827 ,  0.41826412,  0.12784472, -1.4159364 ,  1.557857  ],
       [ 0.44123897,  0.00509824, -0.3819093 ,  0.19909993, -0.56101704,
         0.75481784, -0.8227796 , -0.546978  ,  0.1765861 ,  0.25

In [62]:
mean = weights[0]
sd = weights[1]**0.5

In [63]:
print(mean)
print(sd)

[  3.1303713    3.1304212    3.355535     3.3560905    0.33328012
   0.33343843 305.6317    ]
[0.9357181  0.9362834  1.2527497  1.2537475  0.17972922 0.17982681
 7.2244287 ]


In [64]:
input_gain = 1/sd
input_offset = mean
print(input_gain)
print(input_offset)

[1.0686979  1.0680526  0.79824406 0.7976088  5.5639253  5.560906
 0.13841926]
[  3.1303713    3.1304212    3.355535     3.3560905    0.33328012
   0.33343843 305.6317    ]


In [65]:
num_inputs = len(input_gain)
print(num_inputs)

7


In [40]:
%%gams
SETS
f      'inputs to neural network' /1*7/;

In [66]:
%%gams
SETS
hl1    'number of hidden neurons in layer 1' /1*10/
hl2    'number of hidden neurons in layer 2' /1*7/
hl3    'number of hidden neurons in layer 3' /1*4/
;

In [67]:
%%capture
%run DataTransform.ipynb

In [68]:
%gams_pull f hl1 hl2 hl3

In [69]:
%%gams
parameter input_offset(f), input_gain(f), bias1(hl1), bias2(hl2), bias3(hl3), wt4(hl3), wt1(f<,hl1<), wt2(hl1<,hl2<);
parameter wt3(hl2<,hl3<);
scalar bias4;

In [70]:
print(input_offset)
%gams_push input_offset
%gams display input_offset;
%gams_lst -e
%gams_cleanup -k

[  3.1303713    3.1304212    3.355535     3.3560905    0.33328012
   0.33343843 305.6317    ]
E x e c u t i o n


----    271 PARAMETER input_offset  1-dim Matrix

1   3.130,    2   3.130,    3   3.356,    4   3.356,    5   0.333,    6   0.333,    7 305.632





In [71]:
print(input_gain)
%gams_push input_gain
%gams display input_gain;
%gams_lst -e
%gams_cleanup -k

[1.0686979  1.0680526  0.79824406 0.7976088  5.5639253  5.560906
 0.13841926]
E x e c u t i o n


----    277 PARAMETER input_gain  1-dim Matrix

1 1.069,    2 1.068,    3 0.798,    4 0.798,    5 5.564,    6 5.561,    7 0.138





In [72]:
bias1 = weights[4]
print(bias1.shape)
%gams_push bias1
%gams display bias1;
%gams_lst -e
%gams_cleanup -k

(10,)
E x e c u t i o n


----    283 PARAMETER bias1  1-dim Matrix

1   0.176,    2   0.195,    3   1.450,    4  -1.541,    5  -1.242,    6  -2.167,    7  -0.083,    8  -0.503,    9  -0.417,    10 -1.797





In [73]:
bias4 = weights[10]
bias4 = float(bias4[0])
print(bias4)
print(type(bias4))
%gams_push bias4
%gams display bias4;
%gams_lst -e
%gams_cleanup -k

1.5872334241867065
<class 'float'>
E x e c u t i o n


----    290 PARAMETER bias4                =        1.587  




In [74]:
bias2 = weights[6]
print(bias2.shape)
%gams_push bias2
%gams display bias2;
%gams_lst -e
%gams_cleanup -k

(7,)
E x e c u t i o n


----    296 PARAMETER bias2  1-dim Matrix

1 -0.240,    2  0.507,    3 -0.777,    4 -1.323,    5  2.146,    6  0.318,    7 -1.134





In [75]:
bias3 = weights[8]
print(bias3.shape)
%gams_push bias3
%gams display bias3;
%gams_lst -e
%gams_cleanup -k

(4,)
E x e c u t i o n


----    302 PARAMETER bias3  1-dim Matrix

1 0.147,    2 0.503,    3 0.168,    4 0.219





In [76]:
wt4 = weights[9]
print(wt4)
print(wt4.shape)
wt4 = np.reshape(wt4, (4,))
print(wt4.shape)
%gams_push wt4
%gams display wt4;
%gams_lst -e
%gams_cleanup -k

[[ 4.5366607]
 [-3.7071018]
 [ 8.328455 ]
 [-6.095542 ]]
(4, 1)
(4,)
E x e c u t i o n


----    308 PARAMETER wt4  1-dim Matrix

1  4.537,    2 -3.707,    3  8.328,    4 -6.096





In [77]:
wt1 = weights[3]
print(wt1.shape)
wt1 = pd.DataFrame(index=f,columns=hl1, data=wt1)
print(wt1)
wt1 = gt_from2dim(wt1)
#print(wt1)
%gams_push wt1
%gams display wt1;
%gams_lst -e
%gams_cleanup -k

(7, 10)
          1         2         3         4         5         6         7  \
1  0.487582  0.472731 -0.039272 -1.051675  1.370770  0.875683  0.136681   
2 -2.310114  1.363845 -0.094558  0.733649 -0.425690 -0.319828 -1.347662   
3 -0.802989  2.857594 -0.852168  1.787351 -3.172370 -1.670711 -0.794380   
4  2.440909  1.943513  1.496585 -1.098035  0.454232 -0.790683  0.418264   
5  0.441239  0.005098 -0.381909  0.199100 -0.561017  0.754818 -0.822780   
6 -0.561940  0.295610  0.646172  0.324267  0.085300 -0.304272  0.992068   
7 -0.028596  0.702636  0.103398 -0.018385 -0.037097  0.014223 -0.007244   

          8         9        10  
1  0.971467 -2.137303  0.407233  
2  4.015146  1.268384 -0.581365  
3  0.175578  1.299922 -1.008968  
4  0.127845 -1.415936  1.557857  
5 -0.546978  0.176586  0.253096  
6  0.299085 -0.513587  0.270180  
7 -0.033733 -0.052661 -0.033381  
E x e c u t i o n


----    315 PARAMETER wt1  

            1           2           3           4           5         

In [78]:
wt2 = weights[5]
print(wt2.shape)
wt2 = pd.DataFrame(index=hl1,columns=hl2, data=wt2)
print(wt2)
wt2 = gt_from2dim(wt2)
#print(wt1)
%gams_push wt2
%gams display wt2;
%gams_lst -e
%gams_cleanup -k

(10, 7)
           1         2         3         4         5         6         7
1  -1.282449 -2.009424  1.466542 -1.885752 -1.286529 -1.589981 -2.602633
2  -1.546888  0.939889  1.201288  0.020570  1.522244 -0.005889 -0.131142
3   1.089889  0.528553  0.684630  0.405810 -0.290402 -1.202366 -1.734648
4   2.172801  0.149523 -0.742448 -0.557026 -1.673638  0.010198  2.722237
5   0.527754 -2.761977 -1.906064  1.019415 -1.051341  0.368169 -0.438354
6   1.168246 -0.318318 -0.452387  2.411706  1.173321 -1.483726 -1.264049
7  -0.053928  0.052848 -1.746812  1.464982  2.378031 -1.007449 -1.113941
8  -3.588139 -1.694506  0.465589  0.885197 -1.397662  0.297863 -0.733270
9  -3.252966  0.683201 -0.422933  1.432118 -0.425370 -1.497022  3.271863
10 -1.715387 -1.863428 -1.541181 -1.158269  0.267281  2.045700  0.069875
E x e c u t i o n


----    322 PARAMETER wt2  

             1           2           3           4           5           6           7

1       -1.282      -2.009       1.467      -1.886  

In [79]:
wt3 = weights[7]
print(wt3.shape)
wt3 = pd.DataFrame(index=hl2,columns=hl3, data=wt3)
print(wt3)
wt3 = gt_from2dim(wt3)
#print(wt1)
%gams_push wt3
%gams display wt3;
%gams_lst -e
%gams_cleanup -k

(7, 4)
          1         2         3         4
1  2.116989  0.620980 -0.695346  2.104851
2  4.372233 -1.704406  1.658299 -2.311012
3  3.377651  0.747541  1.697098 -1.134911
4 -2.134733 -1.892926  3.416281 -3.086441
5 -0.687977 -1.591132  0.473572 -1.325456
6 -1.261446  0.062865 -1.687149  0.830625
7  0.836776  1.437133 -2.759117  1.589384
E x e c u t i o n


----    329 PARAMETER wt3  

            1           2           3           4

1       2.117       0.621      -0.695       2.105
2       4.372      -1.704       1.658      -2.311
3       3.378       0.748       1.697      -1.135
4      -2.135      -1.893       3.416      -3.086
5      -0.688      -1.591       0.474      -1.325
6      -1.261       0.063      -1.687       0.831
7       0.837       1.437      -2.759       1.589





In [80]:
%%gams
Free Variables
inp(f)
inter1v(hl1)
inter2v(hl2)
inter3v(hl3)
inter4v
inter1z(hl1)
inter2z(hl2)
inter3z(hl3)
inter4z
z
;

Equations
eq_inter1v(hl1)
eq_inter1z(hl1)
eq_inter2v(hl2)
eq_inter2z(hl2)
eq_inter3v(hl3)
eq_inter3z(hl3)
eq_inter4v
eq_inter4z
eq_z
;

eq_inter1v(hl1)..  					inter1v(hl1) =e= sum(f, wt1(f,hl1)*inp(f)) + bias1(hl1);
eq_inter1z(hl1)..  					inter1z(hl1) =e= 1 - (2/(exp(2*inter1v(hl1))+1));
*eq_inter1z(hl1)..                   inter1z(hl1) =e= exp(-inter1v(hl1)*inter1v(hl1));

eq_inter2v(hl2)..  					inter2v(hl2) =e= sum(hl1, wt2(hl1,hl2)*inter1z(hl1)) + bias2(hl2);
eq_inter2z(hl2)..  					inter2z(hl2) =e= 1 - (2/(exp(2*inter2v(hl2))+1));
*eq_inter2z(hl2)..                   inter2z(hl2) =e= exp(-inter2v(hl2)*inter2v(hl2));

eq_inter3v(hl3)..  					inter3v(hl3) =e= sum(hl2, wt3(hl2,hl3)*inter2z(hl2)) + bias3(hl3);
eq_inter3z(hl3)..  					inter3z(hl3) =e= 1 - (2/(exp(2*inter3v(hl3))+1));
*eq_inter3z(hl3)..                   inter3z(hl3) =e= exp(-inter3v(hl3)*inter3v(hl3));

eq_inter4v..     					inter4v =e= sum(hl3, wt4(hl3)*inter3z(hl3)) + bias4;
eq_inter4z..     					inter4z =e= 1/(1 + exp(-inter4v));
eq_z..                              z =e= inter4z;
model NN /all/;

In [81]:
%gams_cleanup -k

In [82]:
%gams solve NN max inter4v using nlp;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),15.1817,45,52,NLP,CONOPT,0


In [ ]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import LinExpr

In [ ]:
num_inputs = 6
num_neurons1 = 2
num_neurons2 = 1
num_neurons3 = 5
LB1_CP = []
UB1_CP = []
x_lb = -50
x_ub = 50
wt1=weights[3]
b1=weights[4]
wt2=weights[5]
b2=weights[6]
wt3=weights[7]
b3=weights[8]
wt4=weights[9]
b4=weights[10]
for i in range(num_neurons1):
    sum_lb = 0
    sum_ub = 0
    for j in range(num_inputs):
    #print(j,i)
        sum_lb += ((x_lb*max(0,wt1[j][i])) + (x_ub*min(0,wt1[j][i])))
        sum_ub += ((x_ub*max(0,wt1[j][i])) + (x_lb*min(0,wt1[j][i])))
    sum_lb += b1[i]
    sum_ub += b1[i]
    LB1_CP.append(sum_lb)
    UB1_CP.append(sum_ub)
LB2_CP = []
UB2_CP = []
for i in range(num_neurons2):
    sum_lb = 0
    sum_ub = 0
    for j in range(num_neurons1):
        sum_lb += ((LB1_CP[j]*max(0,wt2[j][i])) + (UB1_CP[j]*min(0,wt2[j][i])))
        sum_ub += ((UB1_CP[j]*max(0,wt2[j][i])) + (LB1_CP[j]*min(0,wt2[j][i])))
    sum_lb += b2[i]
    sum_ub += b2[i]
    LB2_CP.append(sum_lb)
    UB2_CP.append(sum_ub)
LB3_CP = []
UB3_CP = []
for i in range(num_neurons3):
    sum_lb = 0
    sum_ub = 0
    for j in range(num_neurons2):
        sum_lb += ((LB2_CP[j]*max(0,wt3[j][i])) + (UB2_CP[j]*min(0,wt3[j][i])))
        sum_ub += ((UB2_CP[j]*max(0,wt3[j][i])) + (LB2_CP[j]*min(0,wt3[j][i])))
    sum_lb += b3[i]
    sum_ub += b3[i]
    LB3_CP.append(sum_lb)
    UB3_CP.append(sum_ub)
print(LB1_CP)
print(UB1_CP)
print(LB2_CP)
print(UB2_CP)
print(LB3_CP)
print(UB3_CP)

In [ ]:
for i in range(0,len(UB1_CP)):
    if UB1_CP[i] < 0:
        UB1_CP[i] = 0
for i in range(0,len(UB2_CP)):
    if UB2_CP[i] < 0:
        UB2_CP[i] = 0  
for i in range(0,len(UB3_CP)):
    if UB3_CP[i] < 0:
        UB3_CP[i] = 0
print(UB1_CP)
print(UB2_CP)
print(UB3_CP)

In [ ]:
def check_isequal(lb, ub):
    length = len(lb)
    check = False
    count = 0
    for i in range(length):
        if lb[i] == ub[i]:
            count += 1
    if count == length:
        check=True
    return check

In [ ]:
def process_bounds(lb,ub):
    length = len(lb)
    LB=[None]*length
    UB=[None]*length
    check=check_isequal(lb,ub)
    if check == True:
        LB = 0
        UB = 0
    else:
        LB = lb
        UB = ub
    return LB, UB

In [ ]:
LB1_CP, UB1_CP = process_bounds(LB1_CP, UB1_CP)
LB2_CP, UB2_CP = process_bounds(LB2_CP, UB2_CP)
LB3_CP, UB3_CP = process_bounds(LB3_CP, UB3_CP)

In [ ]:
%%gams
parameter bigm1lb(hl1), bigm1ub(hl1), bigm2lb(hl2), bigm2ub(hl2), bigm3lb(hl3), bigm3ub(hl3);

In [ ]:
def OBBT(n1, n2, lb1, lb2, ub1, ub2, w, b):
    m = gp.Model("OBBT")
    x = m.addVars (n1, lb=lb1, ub=ub1, vtype=GRB.CONTINUOUS, name="input")
    interz = m.addVars(n2, lb=0, ub=ub2, vtype=GRB.CONTINUOUS, name="interz")
    interv = m.addVars(n2, lb=lb2, ub=ub2, vtype=GRB.CONTINUOUS, name="interv")
    y = m.addVars(n2, vtype=GRB.BINARY)
    linexp = LinExpr()
    for i in range(n2):
        for j in range(n1):
            linexp += LinExpr(w[j][i],x[j])
        linexp += b[i]
    m.addConstr(interv[i] - linexp == 0)
    m.addConstr(interz[i] >= interv[i])
    m.addConstr(interz[i] <= interv[i] - (1 - y[i])*lb2[i])
    m.addConstr(interz[i] <= y[i]*ub2[i])
    UB2 = [None]*n2
    LB2 = [None]*n2
    lb = [None]*n2
    ub = [None]*n2
    for i in range(n2):
        m.setParam('TimeLimit',120)
        m.setParam('LogToConsole',0)
        m.setObjective(interv[i],GRB.MINIMIZE)
        m.optimize()
        if m.getAttr(GRB.Attr.SolCount) != 0:
            if m.getAttr(GRB.Attr.Status) == GRB.OPTIMAL:
                lb[i] = m.getAttr(GRB.Attr.ObjVal)
            else:
                lb[i] = m.getAttr(GRB.Attr.ObjBound)
            if lb[i] > lb2[i]:
                LB2[i] = lb[i]
            else:
                LB2[i] = lb2[i]
                print('Improved lower bound not found!')
        else:
            LB2[i] = lb2[i]
            print('Improved lower bound not found!')
        m.reset()
        m.setObjective(interv[i], GRB.MAXIMIZE)
        m.optimize()
        if m.getAttr(GRB.Attr.SolCount) != 0:
            if m.getAttr(GRB.Attr.Status) == GRB.OPTIMAL:
                ub[i] = m.getAttr(GRB.Attr.ObjVal)
            else:
                ub[i] = m.getAttr(GRB.Attr.ObjBound)
            if ub[i] < ub2[i]:
                UB2[i] = ub[i]
            else:
                UB2[i] = ub2[i]
                print('Improved upper bound not found!') 
        else:
            UB2[i] = ub2[i]
            print('Improved upper bound not found!') 
        #m.computeIIS()
        #m.write("infeas.ilp")
        m.reset()
    return LB2, UB2

In [ ]:
LB1_0, UB1_0 = OBBT(num_inputs, num_neurons1, -50, LB1_CP, 50, UB1_CP, wt1, b1)
print(LB1_CP, UB1_CP)
print(LB1_0, UB1_0)

In [ ]:
LB2_0, UB2_0 = OBBT(num_neurons1, num_neurons2, 0, LB2_CP, UB1_0, UB2_CP, wt2, b2)
print(LB2_CP, UB2_CP)
print(LB2_0, UB2_0)

In [ ]:
LB3_0, UB3_0 = OBBT(num_neurons2, num_neurons3, 0, LB3_CP, UB2_0, UB3_CP, wt3, b3)
print(LB3_CP, UB3_CP)
print(LB3_0, UB3_0)

In [ ]:
bigm1lb = np.array(LB1_0)
bigm1ub = np.array(UB1_0)
bigm2lb = np.array(LB2_0)
bigm2ub = np.array(UB2_0)
bigm3lb = np.array(LB3_0)
bigm3ub = np.array(UB3_0)

In [ ]:
%gams_push bigm1lb
%gams display bigm1lb;
%gams_lst -e
%gams_cleanup -k
%gams_push bigm1ub
%gams display bigm1ub;
%gams_lst -e
%gams_cleanup -k
%gams_push bigm2lb
%gams display bigm2lb;
%gams_lst -e
%gams_cleanup -k
%gams_push bigm2ub
%gams display bigm2ub;
%gams_lst -e
%gams_cleanup -k
%gams_push bigm3lb
%gams display bigm3lb;
%gams_lst -e
%gams_cleanup -k
%gams_push bigm3ub
%gams display bigm3ub;
%gams_lst -e
%gams_cleanup -k